In [1]:
import pandas as pd
import requests
import time

from etl_resources import sqlite_connection, get_api_key, get_symbol_list

In [2]:
def endpoints():
    
    api_key = get_api_key()
    
    _endpoints = {
    'cpi': f"https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey={api_key}",
    'federal_funds': f"https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval=monthly&apikey={api_key}",
    'gdp': f"https://www.alphavantage.co/query?function=REAL_GDP&interval=quarterly&apikey={api_key}",
    'retail_sales': f"https://www.alphavantage.co/query?function=RETAIL_SALES&apikey={api_key}",
    'unemployment': f"https://www.alphavantage.co/query?function=UNEMPLOYMENT&apikey={api_key}"    
    }
    
    return _endpoints

In [3]:
def macro_indicators():
    
    '''
    This function uses the alphavantage api to pull macro indicators
    '''
    
    con = sqlite_connection()

    _endpoints = endpoints()
    
    for table, endpoint in _endpoints.items():
        try:

            print(f"Parsing {table} data")
            response = requests.get(f"{endpoint}")
            json = response.json()["data"]
            df = pd.DataFrame.from_dict(json)

            df.to_sql(name=f"{table}", if_exists='replace', index=False, con=con)

        except:
            # TODO: Better failure logging
            print(f"Failed: {table}")
 


In [4]:
def process_macro_datatypes():
    
    '''
    This function cleans up the datatypes for the 2 fields in these macro indicator tables.
    '''
    
    _endpoints = endpoints()
    con = sqlite_connection()
    
    for table in _endpoints:
        
        print(f"Processing {table}")
        df = pd.read_sql(f"select * from {table}",con=con)
        df['date'] = pd.to_datetime(df['date'])
        df['value'] = pd.to_numeric(df['value'])
        
        df.to_sql(f"{table}_clean",con=con)
        

In [ ]:
def process_timeseries_differences():
    
    _endpoints = endpoints()
    con = sqlite_connection()
    
    for table in _endpoints:
        
        

In [5]:
def main():
    macro_indicators()
    process_macro_datatypes()

In [6]:
main()

Parsing cpi data
Parsing federal_funds data
Parsing gdp data
Parsing retail_sales data
Parsing unemployment data
Processing cpi
Processing federal_funds
Processing gdp
Processing retail_sales
Processing unemployment
